# Proyek Prediksi Risiko Diabetes (Smart Select)

Notebook ini telah diperbarui dengan logika cerdas untuk **memfilter file non-data** (seperti file `Index` atau dokumentasi) yang sering ada di dalam file ZIP dari UCI Repository.

**Dataset:** [UCI Diabetes Dataset (ID: 34)](https://archive.ics.uci.edu/dataset/34/diabetes)

**Langkah Awal:**
Pastikan Anda mengupload file `.zip` yang berisi data diabetes.

## 1. Import Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

# Library upload & ZIP handling
from google.colab import files
import io
import zipfile
import os

## 2. Upload, Ekstrak, dan Memuat Data (Auto-Filter)
Kode di bawah ini akan mengekstrak ZIP, lalu mencari file data yang valid dengan **mengabaikan file bernama 'Index'** atau folder.

In [ ]:
print("Silakan upload file 'diabetes.zip':")
uploaded = files.upload()

zip_filename = next(iter(uploaded))

print(f"\nMengekstrak {zip_filename}...")

data_file_name = None

with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall()
    extracted_files = zip_ref.namelist()
    print(f"File yang ditemukan dalam ZIP: {extracted_files}")
    
    # --- LOGIKA PEMILIHAN FILE OTOMATIS ---
    for f in extracted_files:
        # Lewati jika nama filenya mengandung 'index' (case insensitive) atau itu adalah folder
        if 'index' in f.lower() or f.endswith('/'):
            print(f"-> Mengabaikan file: {f} (Bukan data utama)")
            continue
        
        # Ambil file pertama yang lolos filter (biasanya file .data atau .csv)
        data_file_name = f
        break

if data_file_name:
    print(f"\n[SUKSES] Membaca data dari file target: {data_file_name}")
    
    nama_kolom = [
        'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
        'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
    ]

    # Membaca CSV
    diabetes_dataset = pd.read_csv(data_file_name, header=None, names=nama_kolom)
    
    # Menampilkan data
    display(diabetes_dataset.head())
else:
    print("\n[ERROR] Tidak dapat menemukan file data yang valid dalam ZIP.")

## 3. Analisis Data Singkat

In [ ]:
print(f"Ukuran data: {diabetes_dataset.shape}")
diabetes_dataset.describe()

In [ ]:
print("Jumlah data per label:")
print(diabetes_dataset['Outcome'].value_counts())

## 4. Preprocessing Data (Standarisasi)

In [ ]:
X = diabetes_dataset.drop(columns='Outcome', axis=1)
Y = diabetes_dataset['Outcome']

scaler = StandardScaler()
scaler.fit(X)
standarized_data = scaler.transform(X)

X = standarized_data
print(X[:5])

## 5. Train/Test Split & Modeling

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)

classifier = svm.SVC(kernel='linear')
classifier.fit(X_train, Y_train)

train_acc = accuracy_score(classifier.predict(X_train), Y_train)
test_acc = accuracy_score(classifier.predict(X_test), Y_test)

print(f'Akurasi Training: {train_acc * 100:.2f}%')
print(f'Akurasi Testing : {test_acc * 100:.2f}%')

## 6. Prediksi Data Baru

In [ ]:
input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

input_as_numpy = np.asarray(input_data).reshape(1, -1)
std_data = scaler.transform(input_as_numpy)
prediction = classifier.predict(std_data)

print(f"Prediksi: {'BERISIKO Diabetes' if prediction[0] == 1 else 'TIDAK berisiko'}")